<a id='SimulatedAndExperimentalDataModelTunerTop'></a>
# Hyperameter Tuning for Single LSTM Model Trained on Simulated and Experimental Data Simultaneously using Keras Tuner

Hyperparameter tuning of single LSTM model on a mixture of the experimental and simulated datasets, using keras tuner.

- Tunable parameters are:
    - number of nodes in each layer (5-1000)
    - number of hidden layers (2-9)
    - batch size (2-128)
- Optional tunable parameters are:
    - regularization
    - dropout
    - custom learning rate

Hyperparameter tuning parameters
- max epoch = 2000
- random search
- early stopping after 50 epochs of no improvement, or validation loss is nan

The hyperparameters and validation loss from the epoch and model which resulted in the best validation loss is saved, informing the hyperparameters of the [model](ExperimentalDataEnsembleModels.ipynb#ExperimentalDataEnsembleModelsTop)

Minimum loss is -ln(minimum_variance)/2 = -6.91 (for a minimum variance chosen to be 1e-6)

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from LSTMutils import MeanVarianceLogLikelyhoodLoss
import LSTMutils
from sklearn.model_selection import train_test_split
import keras_tuner as kt

# input parameters
SequenceLength = 250
validation_split = 0.25
NumEpochs = 2000
test_split = 0.2
AddDropout = False
AddRegularization = False
CustomLearningRate = False
EarlyStoppingPatience = 50
NumTopResults = 30

#set random seeds
np.random.seed(42)
tf.random.set_seed(42)

# read experimental dataset
ExperimentalData = LSTMutils.ExperimentalData(SequenceLength=SequenceLength)
time, experimental_concentrations, df_experimental, unused = ExperimentalData.ReadData()

# read simulated dataset
SimulatedData = LSTMutils.ExperimentalData(ExperimentalDataFilePath="../TrainingData/SimulatedTrainingSet10000.csv",SequenceLength=SequenceLength)
unused, unused, df_simulated, unused = SimulatedData.ReadData()

# split data into stratified train and test sets, size defined by the test_split variable
# the split will always be the same provided the data is in the same order, the same random_state is used,
# and strangely the labels used for stratification are always the same type (str is used here)
df_train_experimental, df_test = train_test_split(df_experimental, test_size=test_split, train_size=1-test_split, random_state=42, shuffle=True, stratify=experimental_concentrations)

experimental_train_concentrations = df_train_experimental.iloc[:,0]
df_train_experimental, df_val = train_test_split(df_train_experimental, test_size=validation_split, train_size=1-validation_split, random_state=42, shuffle=True, stratify=experimental_train_concentrations)

# combine simulated dataset and experimental training dataset into one large training set
df_simulated.columns = range(1,df_simulated.columns.size+1)
df_train = pd.concat([df_train_experimental, df_simulated], ignore_index=True)

# normalise time series data
df_norm_train, df_norm_test, df_norm_val = SimulatedData.NormalizeData(df_train,df_test,df_val)

# randomly shuffle data
df_norm_train = df_norm_train.sample(frac=1)
df_norm_val = df_norm_val.sample(frac=1)
    
# Define y as the last element in X, and ensure X and y are the correct shape
X_train, y_train = SimulatedData.Shape(df_norm_train)
X_val, y_val = SimulatedData.Shape(df_norm_val)

# implementation informed by https://github.com/keras-team/keras-tuner/issues/122
# set up class to enable batch size to be searched with the rest of the hyperparameters
class LSTMHyperModel(kt.HyperModel):
    
    def build(self,hp):
        
        # if needed, add L2 regularizer
        if AddRegularization:
            Regularizer = keras.regularizers.L2(hp.Choice("L2Regularizer1", [0., 1e-4]))
        else:
            Regularizer = None        
        
        # define network architecture of input and first hidden layer
        model = keras.models.Sequential(keras.layers.LSTM(hp.Choice("layer_1_units", [5,10,20,50,100,200,500,1000,2000])
                                                          , input_shape=(SequenceLength,1)
                                                          , return_sequences=True, kernel_regularizer=Regularizer))
        
        # if needed add dropout
        if AddDropout:
            if hp.Boolean(f"layer_{i+2}_dropout"):
                model.add(keras.layers.Dropout(rate=0.25))
                                        
        # define network architecture of subsequent hidden layers                                        
        for i in range(hp.Int("LSTM Layers", min_value = 1, max_value = 8)):
            model.add(keras.layers.LSTM(hp.Choice(f"layer_{i+2}_units", [5,10,20,50,100,200,500,1000,2000])
                                        , return_sequences=True, kernel_regularizer=Regularizer))
            
            # if needed add dropout
            if AddDropout:
                if hp.Boolean(f"layer_{i+2}_dropout"):
                    model.add(keras.layers.Dropout(rate=0.25))
                                        
        # if needed specify learning rate schedule            
        if CustomLearningRate:
            learning_rate = hp.Float("lr", min_value=1e-5, max_value=1e-2, sampling="log")

        # add output layer with softmax activation to ensure variance predictions are positive
        model.add(keras.layers.LSTM(2, activation='softplus',return_sequences=True))

        # adam optimizer and custom negative log likelyhood loss
        model.compile(optimizer="adam",loss = MeanVarianceLogLikelyhoodLoss)
    
        return model

    # define custom fit function called by keras tuner that includes batch size as a hyperparameter                          
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(*args,batch_size=hp.Choice("batch_size",[32,64,128,256,512,1024]),**kwargs)

# set up tuner
tuner = kt.RandomSearch(
    LSTMHyperModel(),
    objective='val_loss',
    overwrite = True,
    max_trials = 300,
    seed = 42,
    directory = "../Models/SimulatedAndExperimentalModelTuner")
                      
# run random hyperparameter search, with early stopping if there is no decrease in validation loss
# in 'EarlyStoppingPatience' epochs, or the loss is nan 
tuner.search(X_train, y_train, epochs=NumEpochs, validation_data=(X_val,y_val), callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',patience=50),keras.callbacks.TerminateOnNaN()])

# get and evaluate model from the epoch that achieved the best validation loss
best_model = tuner.get_best_models()[0]
best_model.summary()
best_model.save("../Models/SimulatedAndExperimentalModelTuner")

# show summary of top 'NumTopResults' candidate sets of hyperparameters, ordered in terms of lowest validation loss
tuner.results_summary(NumTopResults)

Trial 3 Complete [00h 00m 32s]
val_loss: -1.2848880290985107

Best val_loss So Far: nan
Total elapsed time: 00h 00m 52s

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
50                |10                |layer_1_units
3                 |1                 |LSTM Layers
1000              |500               |layer_2_units
512               |32                |batch_size
1000              |None              |layer_3_units
1000              |None              |layer_4_units
500               |None              |layer_5_units
10                |None              |layer_6_units
10                |None              |layer_7_units
1000              |None              |layer_8_units

Epoch 1/2


2024-01-04 15:25:27.276617: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.38GiB (rounded to 2560000000)requested by op CudnnRNN
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-01-04 15:25:27.276719: I tensorflow/core/common_runtime/bfc_allocator.cc:1027] BFCAllocator dump for GPU_0_bfc
2024-01-04 15:25:27.276735: I tensorflow/core/common_runtime/bfc_allocator.cc:1034] Bin (256): 	Total Chunks: 93, Chunks in use: 92. 23.2KiB allocated for chunks. 23.0KiB in use in bin. 2.4KiB client-requested in use in bin.
2024-01-04 15:25:27.276745: I tensorflow/core/common_runtime/bfc_allocator.cc:1034] Bin (512): 	Total Chunks: 6, Chunks in use: 6. 3.0KiB allocated for chunks. 3.0KiB in use in bin. 1.9KiB client-requested in use in bin.
2024-01-04 15:25:27.276754: I tens

KeyboardInterrupt: 